In [1]:
import cv2
import numpy as np
import math
from itertools import *
import os

In [2]:
class Preprocessing:
    
    def __init__(self, path):
        
        self.img = np.full((160, 96), 0)
        self.img_array = np.full((160, 96), 0)
        self.gray = np.full((160, 96), 0)
        self.gray_array = np.full((160, 96), 0)
        
        self.G_X = np.full((160, 96), 0)
        self.G_Y = np.full((160, 96), 0)
        self.g_x = np.full((160, 96), 0)
        self.g_y = np.full((160, 96), 0)
        self.g_mag = np.full((160, 96), 0)
        self.sobel_image = np.full((160, 96), 0)
        
        self.theta = np.full((160, 96), 0)       
        
        self.path = path
        self.img_array = np.full((160, 96), 0)
        self.image_read()
        self.gray = self.rgb2gray(self.img_array)
        self.image_write(self.gray)
        
        self.g_x, self.g_y, self.sobel_output = np.round(self.sobels_operator(self.gray))
        self.theta = self.gradient_angles(self.gray_array, self.g_x, self.g_y)
#         print(self.g_x[28][28])
#         print(self.g_y[28][28])
#         print(self.theta[28][28])
        
#         return  self.img_array, self.gray_array, self.g_mag, self.theta
        
#         self
    
    def image_read(self):
        self.img = cv2.imread(self.path)  
        self.img_array = np.array(self.img, dtype=float)
#         self.gray = self.rgb2gray(self.img)
#         self.
    
    def rgb2gray(self, rgb):
        return np.round(np.dot(rgb[...,:3], [0.299, 0.587, 0.114]))
    
    def image_write(self, gray):
        self.img_gray = cv2.imwrite('C:/Users/anvit/OneDrive/Desktop/HoG/Test_results/test_gray.bmp', self.gray)
        self.gray_array = np.array(gray, dtype=float)
#         return self.gray_array

    #sobels operator
    def sobels_operator(self, inp):
        self.inp = inp
        g_x = np.array([[-1,0,1],
                       [-2,0,2],
                       [-1,0,1]])
        g_y = np.array([[1,2,1],
                       [0,0,0],
                       [-1,-2,-1]])
    
        #Horizontal Gradient 
        self.G_X = self.conv_sobel(self.inp,g_x)
        #Saving the image
        cv2.imwrite('C:/Users/anvit/OneDrive/Desktop/HoG/Test_results/test_gray_sobelGX.bmp', self.G_X)
       
        #Vertical Gradient
        self.G_Y = self.conv_sobel(self.inp,g_y)
        #saving the image
        cv2.imwrite('C:/Users/anvit/OneDrive/Desktop/HoG/Test_results/test_gray_sobelGY.bmp', self.G_Y)
    
        #Gradient Magnitude
        self.g_mag = np.sqrt(np.square(self.G_X) + np.square(self.G_Y))
        #saving the image
        cv2.imwrite('C:/Users/anvit/OneDrive/Desktop/HoG/Test_results/test_gray_sobel.bmp', self.g_mag)    
        return self.G_X, self.G_Y, self.g_mag

    #convolution done by implementing using 2D directly
    def conv_sobel(self,x,y):
        
        row_size = x.shape[0]
        column_size = x.shape[1]        
        conv_img = np.zeros((row_size, column_size), dtype=np.float)
        
        for a in range(0, row_size - 2):
            for b in range(0, column_size - 2):
                
                conv_img[a + 1][b + 1] = (np.sum(x[a: a + 3, b: b + 3] * y)) / 4

        return conv_img
    
    def gradient_angles(self, gray_array, g_x, g_y):
        #Finding the Gradient Angles i.e., theta
        print(self.gray_array.shape[0])
        #arctan2 is tan inverse, which gives us results in radians with [-pi to pi]
        self.theta = np.zeros((self.gray_array.shape[0],self.gray_array.shape[1]))
        for i in range(self.gray_array.shape[0]):
            for j in range(0, self.gray_array.shape[1]):
                if(self.g_y[i][j] == 0 and self.g_x[i][j] == 0):
                    self.theta[i][j] = 0
                elif(self.g_y[i][j] > 0 and self.g_x[i][j] == 0):
                    self.theta[i][j] = 90
                elif(self.g_y[i][j] < 0 and self.g_x[i][j] == 0):
                    self.theta[i][j] = -90
                elif(self.g_y[i][j] == 0):
                    self.theta[i][j] = 0
                else:
                    if(self.g_x[i][j] == 0):
                        print('here')
                    self.theta[i][j] = np.arctan2(self.g_y[i][j],self.g_x[i][j])
    
        #Converting radians to degrees
        self.theta = np.rad2deg(self.theta)
        #converting all the negatives into positives by adding 360 so the range is now [0 to 360]
        for i in range(0, self.gray_array.shape[0]):
            for j in range(0, self.gray_array.shape[1]):
                if(self.theta[i][j] < 0):
                    self.theta[i][j] += 360
                    
        return self.theta        

In [3]:
class Histogram:
    def __init__(self, mag_block, theta_block):
        self.center = np.array([0,20,40,60,80,100,120,140,160])
#         4 histograms of 9 bins each - per block
        self.bins = np.zeros((4,9), dtype=float)
        self.cell_size = 8
        self.cells = []
        self.t_cells = []
        self.m_cells = []
        self.flattened = []
        self.theta_block = theta_block
        self.mag_block = mag_block
        self.convert_block_2_cell(mag_block,theta_block)
#         c=0
#         print(c)
        self.generate_features(self.m_cells, self.t_cells)
        self.norm_hog()
        self.flattened_bins()
        
    def flattened_bins(self):
        for bin in self.bins:
            for item in bin:
                self.flattened.append(item)
        
    
    def norm_hog(self):
        
        summation = 0
        for bin in self.bins:
            for j in range(len(bin)):
                summation += bin[j] ** 2
        
        dist = summation ** 0.5
                   
        for bin in self.bins:
            for j in range(len(bin)):
                if(dist == 0):
                    continue
                else:
                    bin[j] /= dist

    
    def convert_block_2_cell(self, mag_block, theta_block):
        c=0
        for m in range(0,theta_block.shape[0], self.cell_size):
                for n in range(0, theta_block.shape[1], self.cell_size):
                    
                    m_cell = np.zeros((self.cell_size,self.cell_size))
                    t_cell = np.zeros((self.cell_size,self.cell_size))
                    
                    for p in range(self.cell_size):
                        for q in range(self.cell_size):
                            
                            m_cell[p][q] = mag_block[m+p][n+q]
                            t_cell[p][q] = theta_block[m+p][n+q]
                     
                    c+=1
                    self.m_cells.append(m_cell)
                    self.t_cells.append(t_cell)
#         print(c)
    
    def generate_features(self, m_cells, t_cells):
        
        for i in range(len(t_cells)):
            for j in range(len(t_cells[i])):
                for k in range(len(t_cells[i][j])):
                    self.cal_hist(m_cells[i][j][k],t_cells[i][j][k], self.bins[i]) 
                
# HANDLE EDGE CASE : -20,10       
    def cal_hist(self, mag, angle, bins):       
        
#         COMPUTE THE DISTANCE TO ADD THE MAG WRT THE CENTER
        if(angle >= 180):
            angle -= 180
#         now we have all unsigned angles
# new case to consider: if angle is >=160 first = bin 9 and second is bin 1
        
        if(angle >= 160):
            first_bin = 8
            second_bin = 0
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag 
#         edge care , if angle is >=350 it's between bins 1 and 9
        
        if(angle >= 0 and angle < 20):
            first_bin = 0
            second_bin = 1
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
        
        if(angle >= 20 and angle < 40):
            first_bin = 1
            second_bin = 2
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
            
        if(angle >= 40 and angle < 60):
            first_bin = 2
            second_bin = 3
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
            
        if(angle >= 60 and angle < 80):
            first_bin = 3
            second_bin = 4
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
        
        if(angle >= 80 and angle < 100):
            first_bin = 4
            second_bin = 5
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
        
        if(angle >= 100 and angle < 120):
            first_bin = 5
            second_bin = 6
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
        
        if(angle >= 120 and angle < 140):
            first_bin = 6
            second_bin = 7
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
         
        if(angle >= 140 and angle < 160):
            first_bin = 7
            second_bin = 8
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
            
    def calc_distance(self, angle, center):        
        percent = (np.absolute(angle - center))/20
        return percent

In [4]:
def h_feature(mag, theta): 
    descriptor = []
    cell_size = 8
    block_size = 16
    block_overlap = 8
    assert(mag.shape[0] == theta.shape[0])
    assert(mag.shape[1] == theta.shape[1])
    b=0
    c=0
    
#     generating blocks from pixels
    for i in range(0,theta.shape[0]-cell_size, cell_size):
        for j in range(0,theta.shape[1]-cell_size, cell_size):
            
            m_block = np.zeros((block_size,block_size))
            t_block = np.zeros((block_size,block_size))
            
            for k in range(block_size):
                for l in range(block_size):  
                    
                    m_block[k][l] = mag[i+k][j+l]
                    t_block[k][l] = theta[i+k][j+l]
            b +=1

            hist_obj = Histogram(m_block,t_block)
            for item in hist_obj.flattened:
                descriptor.append(item)
#     print(b)
#     print(c)
    return descriptor


In [5]:
# h_feature_output = np.array(h_feature(sobel_output,theta))
# print(np.concatenate(h_feature_output).ravel().tolist())
# print(h_feature_output)

In [6]:
class LBP:
    def __init__(self, gray_block):
        self.mag = gray_block
        self.bins = np.zeros((59,), dtype=float)
        self.generate_features(gray_block)
        self.norm_lbp()
        
    def norm_lbp(self):
        for i in range(len(self.bins)):
            self.bins[i] = self.bins[i] / 256
    
    def generate_features(self,gray_block):
        for i in range(gray_block.shape[0]):
            for j in range(gray_block.shape[1]):
                self.compute_LBP_pattern(i,j,gray_block)
        
#     def norm_lbp(self):
#         for i in range(len(self.bins)):
#             self.bins[i] = self.bins[i] / 256
        
    def compute_LBP_pattern(self,i, j,gray_block):
        x = gray_block
        pattern = ''
        
        try:
            if(x[i-1][j-1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            if(x[i-1][j] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            if(x[i-1][j+1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            if(x[i][j+1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            if(x[i+1][j+1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
            
            if(x[i+1][j] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
            
            if(x[i+1][j-1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
            
            if(x[i][j-1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            self.append_to_bins(pattern)
            
        except IndexError:
            pattern = '00000101'
            self.append_to_bins(pattern)
        
        
        
    def append_to_bins(self, lbp_pattern):        
        decimal = int(lbp_pattern, 2)
        patterns = {
            0: 0,
            1: 1,
            2: 2,
            3: 3,
            4: 4,
            6: 5,
            7: 6,
            8: 7,
            12: 8,
            14: 9,
            15: 10,
            16: 11,
            24: 12,
            28: 13,
            30: 14,
            31: 15,
            32: 16,
            48: 17,
            56: 18,
            60: 19,
            62: 20,
            63: 21,
            64: 22,
            96: 23,
            112: 24,
            120: 25,
            124: 26,
            126: 27,
            127: 28,
            128: 29,
            129: 30,
            131: 31,
            135: 32,
            143: 33,
            159: 34,
            191: 35,
            192: 36,
            193: 37,
            195: 38,
            199: 39,
            207: 40,
            223: 41,
            224: 42,
            225: 43,
            227: 44,
            231: 45,
            239: 46,
            240: 47,
            241: 48,
            243: 49,
            247: 50,
            248: 51,
            249: 52,
            251: 53,
            252: 54,
            253: 55,
            254: 56,
            255: 57
        }
        bin_number = patterns.get(decimal, 58)
        self.bins[bin_number] += 1   
        
    

In [7]:
def LBP_feature(gray):
    lbp_descriptor = []
    cell_size = 8
    block_size = 16
    block_overlap = 8
    b=0
#     c=0
    
#     generating blocks from pixels   
    for i in range(0,gray.shape[0],block_size):
        for j in range(0,gray.shape[1],block_size):
            
            gray_block = np.zeros((block_size,block_size))
            
            for p in range(block_size):
                for q in range(block_size):
                    
                    gray_block[p][q] = gray[i+p][j+q]
            
            
            b+=1
            lbp_obj1 = LBP(gray_block)
            lbp_descriptor.append(lbp_obj1.bins)
#     print(b)        
    return lbp_descriptor
    

In [8]:
# lbp_output = LBP_feature(gray)

In [9]:
# print(len(lbp_output))

In [10]:
class HOG_NeuralNetwork:
    def __init__(self, human_feature_vectors, non_human_feature_vectors, epochs = 3, alpha = 0.01, input_layer_neurons = 7524, hidden_layer_neurons = 200, output_layer_neurons = 1):
        
        self.epochs = epochs
        self.alpha = alpha
        self.input_layer_neurons = input_layer_neurons
        self.hidden_layer_neurons = hidden_layer_neurons
        self.output_layer_neurons = output_layer_neurons
        
#         self.output_hidden_layer = np.full((2, 1), 0.0)
#         self.pred_output = np.full((, 1), 0.0)
#         self.err = np.full((2, 1), 0.0)
        
        #assigning random weights
#         self.hidden_layer_weights = np.random.randn(-0.5, 0.5, size = (self.input_layer_neurons,self.hidden_layer_neurons))
#         self.output_layer_weights = np.random.randn(-0.5, 0.5, size = (self.hidden_layer_neurons,self.output_layer_neurons))
        self.hidden_layer_weights =  (np.random.random_sample((self.input_layer_neurons,self.hidden_layer_neurons)) -0.5) * 2 * self.alpha
        self.output_layer_weights =  (np.random.random_sample((self.hidden_layer_neurons,self.output_layer_neurons)) - 0.5) * 2 * self.alpha
    
        #assigning random bias       
        self.hidden_layer_bias = np.full((1,self.hidden_layer_neurons), -1.0, dtype = float)
        self.output_layer_bias = np.full((1,self.output_layer_neurons), -1.0, dtype = float)
        
#         self.human_feature_vectors = np.array([[0,0],[0,1],[1,0], [1,1]])
        self.human_feature_vectors = human_feature_vectors
        self.non_human_feature_vectors = non_human_feature_vectors
        
        self.output_hidden_layer = None
        self.pred_output = None
        self.pred_output_delta = None
        self.hidden_layer_output_delta = None
        
        
        print(self.non_human_feature_vectors.shape)
        self.train_nn()
        
    # Sigmoid function for the output neuron 
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
        
    # Derivative of Sigmoid in terms of itself:
    def derivative_of_sigmoid(self, x):
        return x * (1 - x)
        
    # ReLu function for the hidden neurons
    def relu(self, x):
        y = x.copy()
        y[x<0] = 0
        return y
        
    # Derivative of ReLU in terms of itself is:    
    def derivative_of_relu(self, x):
        y = x.copy()
        y[x>0] = 1
        y[x<=0] = 0
        return y

    def feedforward(self, inputs):
        
        self.inputs = inputs
#         h_layer_a = np.dot(self.inputs,self.hidden_layer_weights) + self.hidden_layer_bias
        self.output_hidden_layer = self.relu(np.dot(self.inputs,self.hidden_layer_weights) + self.hidden_layer_bias)

#         o_layer_a = np.dot(self.output_hidden_layer, self.output_layer_weights) + self.output_layer_bias
        
        self.pred_output = self.sigmoid(np.dot(self.output_hidden_layer, self.output_layer_weights) + self.output_layer_bias)
        
    def backpropogation(self, target_output):
        
        final_error = target_output - self.pred_output
#         print('final error', final_error)
        self.err = np.absolute(target_output - self.pred_output).sum()
        self.pred_output_delta = final_error * self.derivative_of_sigmoid(self.pred_output)
        
        h_layer_error = self.pred_output_delta.dot(self.output_layer_weights.T)
        self.hidden_layer_output_delta = h_layer_error * self.derivative_of_relu(self.output_hidden_layer)
            
    def update(self, inputs):
            
        #Updating weights and bias
#         self.inputs = inputs
        self.output_layer_weights = self.output_layer_weights + self.output_hidden_layer.T.dot(self.pred_output_delta) * self.alpha
        self.output_layer_bias = self.output_layer_bias + np.sum(self.pred_output_delta,axis = 1, keepdims=True) * self.alpha
    
        self.hidden_layer_weights = self.hidden_layer_weights + inputs.T.dot(self.hidden_layer_output_delta) * self.alpha
        self.hidden_layer_bias = self.hidden_layer_bias + np.sum(self.hidden_layer_output_delta, axis = 1, keepdims=True) * self.alpha

        
    def train_nn(self):
        
        avg_error = 0
        for _ in range(self.epochs):
            target_output = np.array([[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0]]) 
            self.feedforward(self.human_feature_vectors)
            self.backpropogation(target_output)
            self.update(self.human_feature_vectors)
#             if(np.absolute(avg_error - self.err)<= 0.1):
#                 break
            avg_error += self.err
#             if(avg_error <= 0.1):
#                 break
        print('Predicted output after training Negative Images:\n', self.pred_output)
        print('Average Error:\n', avg_error/len(self.human_feature_vectors))
        
        #save anything from training positive here         
        
        # reset values for negative features

#         #assigning random weights
#         self.hidden_layer_weights = np.random.uniform(-0.5, 0.5, size = (self.input_layer_neurons,self.hidden_layer_neurons))
#         self.output_layer_weights = np.random.uniform(-0.5, 0.5, size = (self.hidden_layer_neurons,self.output_layer_neurons))

#         #assigning random bias       
#         self.hidden_layer_bias = np.random.uniform(-1, -1, size = (1,self.hidden_layer_neurons))
#         self.output_layer_bias = np.random.uniform(-1, -1, size = (1,self.output_layer_neurons))

#         self.pred_output = 0
        
# #         self.pred_output = np.full((2, 1), 0.0)
        
#         avg_error = 0
#         for _ in range(self.epochs):
#             target_output = np.full((10,1),0.0)
#             self.feedforward(self.non_human_feature_vectors)
#             self.backpropogation(target_output)
#             self.update(self.non_human_feature_vectors)
#             avg_error += self.err
        
#         print('Predicted output after training Negative Images:\n', self.pred_output)
#         print('Average Error:\n', avg_error/len(self.non_human_feature_vectors))
        
# #         save anything from training negative here 

In [21]:
class Driver:
    if __name__ == '__main__':
        
        
        train_path_positive = "C://Users//anvit//OneDrive//Desktop//HoG//Training_images(Pos)"
        train_path_negative = "C://Users//anvit//OneDrive//Desktop//HoG//Training_images(Neg)"
        test_neg = "C://Users//anvit//OneDrive//Desktop//HoG//Test_images(Neg)"
        test_pos = "C://Users//anvit//OneDrive//Desktop//HoG//Test_images(Pos)"
        
        def gen_hog_vectors(path):
            vectors = []
            files = os.listdir(path)
            for file in files:
                print(file)
                t = Preprocessing(path + '//' + file)
                image = t.img_array
#         print(image)
                gray = t.gray_array
#         print(gray)
                g_x = t.g_x
#         print(g_x[28][28])
                g_y = t.g_y
#         print(g_y[28][28])
                g_mag = t.g_mag
#         print(g_mag[28][28])
                theta = t.theta
#         print(theta[28][28])
                h_feature_output = np.array(h_feature(g_mag,theta))
#                 print(h_feature_output.shape)
#                 h_feature_output = h_feature(g_mag,theta)
                vectors.append(h_feature_output)
                result = np.array(vectors)
#                 print(result)
            return result
        
        def train_nn(inputs, target_output):
            
            epochs = 10
            alpha = 0.05
            input_layer_neurons = 7524
            hidden_layer_neurons = 200
            output_layer_neurons = 1
            err = 0
            
#             inputs = np.array([[0,0],[0,1],[1,0], [1,1]])
#             target_output = np.array([[0],[1],[1],[0]]) 
            
            # Step 1: Initialize the weights and biases to have random values between −0.5 and 0.5:
#             weights
            hidden_layer_weights = np.random.uniform(-0.5,0.5, size = (input_layer_neurons,hidden_layer_neurons)) * 2 * alpha
            output_layer_weights = np.random.uniform(-0.5,0.5, size = (hidden_layer_neurons,output_layer_neurons)) * 2 * alpha

            # bias:
            hidden_layer_bias = np.random.uniform(-1.0, -1.0,size = (1,hidden_layer_neurons))
            output_layer_bias = np.random.uniform(-1.0, -1.0,size = (1,output_layer_neurons))

#             print('Initial Output Layer Weights:\n', output_layer_weights)
#             print('Initial Hidden Layer Weights:\n', hidden_layer_weights)
#             print('Initial Output Layer Bias:\n', output_layer_bias)
#             print('Initial Hidden Layer Bias:\n', hidden_layer_bias)
                
            # Step 2: Training the inputs

            # Step 2a:
            # Setting the corresponding activation(a1) for input_layer
            def sigmoid(x):
                return 1/(1 + np.exp(-x))

            # We need the derivative of this for further calculations
            def derivative_of_sigmoid(x):
            #     derivative of sigmoid in terms of itslef is:
                return x * (1 - x)
            
            def relu(x):
                y = x.copy()
                y[x<0] = 0
                return y
        
            # Derivative of ReLU in terms of itself is:    
            def derivative_of_relu(x):
                y = x.copy()
                y[x>0] = 1
                y[x<=0] = 0
                return y

            for i in range(epochs):
                #     Step 2b:
                #     Feedforward
                h_layer_a = np.dot(inputs, hidden_layer_weights)
                h_layer_a = h_layer_a + hidden_layer_bias
                output_hidden_layer = relu(h_layer_a)
    
                o_layer_a = np.dot(output_hidden_layer, output_layer_weights)
                o_layer_a = o_layer_a + output_layer_bias
                pred_output = sigmoid(o_layer_a)

                #     Step 2c:
                #     Output error
                
                final_error = target_output - pred_output
                err = err + np.absolute(target_output - pred_output).sum()
                pred_output_delta = final_error * derivative_of_sigmoid(pred_output)

                #     Step 2d:
                #     Backpropogating the error
                h_layer_error = pred_output_delta.dot(output_layer_weights.T)
                hidden_layer_output_delta = h_layer_error * derivative_of_relu(output_hidden_layer)

                #     Step 2e:
                #     Updating weights and bias
                output_layer_weights = output_layer_weights + output_hidden_layer.T.dot(pred_output_delta) * alpha
                output_layer_bias = output_layer_bias + np.sum(pred_output_delta) * alpha
    
                hidden_layer_weights = hidden_layer_weights + inputs.T.dot(hidden_layer_output_delta) * alpha
                hidden_layer_bias = hidden_layer_bias + np.sum(hidden_layer_output_delta) * alpha

            print('Final Output Layer Weights:\n', output_layer_weights)
            print('Final Hidden Layer Weights:\n', hidden_layer_weights)
            print('Final Output Layer Bias:\n', output_layer_bias)
            print('Final Hidden Layer Bias:\n', hidden_layer_bias)

            print('Target Output:\n', target_output)
            print('After Training:\n', pred_output)
            for i in range(pred_output.shape[0]):
                if(pred_output[i] >= 0.6):
                    print('Human detected and the predicted output is', pred_output[i])
                if(pred_output[i] > 0.4 and pred_output[i] < 0.6):
                    print('Boarderline human detected and the predicted output is', pred_output[i])
                if(pred_output[i] <= 0.4):
                    print('No human detected and the predicted output is', pred_output[i])
            print('Average error = ', err/ (inputs.shape[0]))
            return hidden_layer_weights, output_layer_weights,hidden_layer_bias,output_layer_bias
                    
        def test_nn(inputs,target_output,hidden_layer_weights, output_layer_weights,hidden_layer_bias,output_layer_bias):
            
            epochs = 10
            alpha = 0.05
            input_layer_neurons = 7524
            hidden_layer_neurons = 200
            output_layer_neurons = 1
            err = 0
            
#             inputs = np.array([[0,0],[0,1],[1,0], [1,1]])
#             target_output = np.array([[0],[1],[1],[0]]) 
            
            # Step 1: Initialize the weights and biases to have random values between −0.5 and 0.5:
            # weights
#             hidden_layer_weights = np.random.uniform(-0.5,0.5, size = (input_layer_neurons,hidden_layer_neurons)) * 2 * 0.01
#             output_layer_weights = np.random.uniform(-0.5,0.5, size = (hidden_layer_neurons,output_layer_neurons)) * 2 * 0.01

#             # bias:
#             hidden_layer_bias = np.random.uniform(-1.0, -1.0,size = (1,hidden_layer_neurons))
#             output_layer_bias = np.random.uniform(-1.0, -1.0,size = (1,output_layer_neurons))

            print('Initial Output Layer Weights:\n', output_layer_weights)
            print('Initial Hidden Layer Weights:\n', hidden_layer_weights)
            print('Initial Output Layer Bias:\n', output_layer_bias)
            print('Initial Hidden Layer Bias:\n', hidden_layer_bias)
                
            # Step 2: Training the inputs

            # Step 2a:
            # Setting the corresponding activation(a1) for input_layer
            def sigmoid(x):
                return 1/(1 + np.exp(-x))

            # We need the derivative of this for further calculations
            def derivative_of_sigmoid(x):
            #     derivative of sigmoid in terms of itslef is:
                return x * (1 - x)
            
            def relu(x):
                y = x.copy()
                y[x<0] = 0
                return y
        
            # Derivative of ReLU in terms of itself is:    
            def derivative_of_relu(x):
                y = x.copy()
                y[x>0] = 1
                y[x<=0] = 0
                return y

#             for i in range(epochs):
                #     Step 2b:
                #     Feedforward
            h_layer_a = np.dot(inputs, hidden_layer_weights)
            h_layer_a = h_layer_a + hidden_layer_bias
            output_hidden_layer = relu(h_layer_a)
    
            o_layer_a = np.dot(output_hidden_layer, output_layer_weights)
            o_layer_a = o_layer_a + output_layer_bias
            pred_output = sigmoid(o_layer_a)
            
            print('Target Output:\n', target_output)
            print('After Training:\n', pred_output)
            for i in range(pred_output.shape[0]):
                if(pred_output[i][0] >= 0.6):
                    print('Human detected and the predicted output is', pred_output[i])
                if(pred_output[i][0] > 0.4 and pred_output[i][0] < 0.6):
                    print('Boarderline human detected and the predicted output is', pred_output[i])
                if(pred_output[i][0] <= 0.4):
                    print('No human detected and the predicted output is', pred_output[i])
#             print('Average error = ', err/ (inputs.shape[0]))
        
        positive_hog_feature_vectors = gen_hog_vectors(train_path_positive)
        target_output_pos = np.full((10,1), 1.0, dtype = float)
        hidden_layer_weights, output_layer_weights,hidden_layer_bias,output_layer_bias = train_nn(positive_hog_feature_vectors, target_output_pos)
        
        positive_test_features = gen_hog_vectors(test_pos)
        target_output_test_pos = np.full((5,1), 1.0, dtype = float)
        pos_test = test_nn(positive_test_features, target_output_test_pos, hidden_layer_weights, output_layer_weights,hidden_layer_bias,output_layer_bias)
        
#         negative_hog_feature_vectors = gen_hog_vectors(train_path_negative)
#         target_output_neg = np.full((10,1), 0.0, dtype = float)
#         hidden_layer_weights, output_layer_weights,hidden_layer_bias,output_layer_bias = train_nn(negative_hog_feature_vectors, target_output_neg)

#         negative_test_features = gen_hog_vectors(test_neg)
#         target_output_test_neg = np.full((5,1), 0.0, dtype = float)
#         neg_test = test_nn(negative_test_features, target_output_test_neg, hidden_layer_weights, output_layer_weights,hidden_layer_bias,output_layer_bias)
        



crop001008b.bmp
160
crop001028a.bmp
160
crop001030c.bmp
160
crop001045b.bmp
160
crop001047b.bmp
160
crop001063b.bmp
160
crop001275b.bmp
160
crop001672b.bmp
160
crop_000010b.bmp
160
person_and_bike_026a.bmp
160
Final Output Layer Weights:
 [[-0.03506396]
 [-0.00945494]
 [ 0.04355367]
 [-0.02548948]
 [-0.01523334]
 [ 0.03467115]
 [-0.00732109]
 [-0.02632876]
 [-0.04014109]
 [-0.00907045]
 [ 0.00594449]
 [-0.01356937]
 [ 0.0418284 ]
 [ 0.00450519]
 [ 0.02377216]
 [-0.03800691]
 [-0.04448621]
 [-0.03560071]
 [ 0.02966682]
 [-0.04956975]
 [ 0.03344621]
 [-0.0005789 ]
 [ 0.02011277]
 [-0.03317486]
 [ 0.0215322 ]
 [-0.03672639]
 [ 0.01887685]
 [-0.03639734]
 [ 0.00241343]
 [ 0.0313198 ]
 [ 0.03629281]
 [ 0.01606039]
 [ 0.01392107]
 [-0.0474192 ]
 [-0.02710417]
 [-0.01672782]
 [-0.01284441]
 [-0.00878597]
 [ 0.00518651]
 [-0.0209241 ]
 [ 0.05145679]
 [ 0.03150066]
 [-0.02942597]
 [-0.04492624]
 [ 0.03588561]
 [-0.00310313]
 [-0.03069692]
 [ 0.00185058]
 [-0.02042786]
 [ 0.03420516]
 [ 0.063561

In [12]:
np.random.rand(3,2)*0.01

array([[0.00126032, 0.0082592 ],
       [0.00473213, 0.00837586],
       [0.00664615, 0.00044849]])

In [13]:
 (0.5 - (-0.5))* np.random.random_sample((3, 2)) - 0.5

array([[ 0.49141856,  0.31558122],
       [-0.28049982, -0.41756153],
       [ 0.34165949, -0.33732752]])